In [16]:
import numpy as np
import pandas as pd
import ANFIS.load_weights as load_weights
import torch
import re
import glob
from IPython.display import Markdown, display

In [2]:
def daily_format(df):
    df = df[df['period'] == 0]
    daily = pd.DataFrame(columns=['Day', 'TankTemp', 'ClosingStock_tc_readjusted', 'ClosingHeight_tc_readjusted', 'Var_tc_readjusted'])
    for day, day_group in df.groupby(df['Time'].dt.date):
        last_status = day_group.iloc[-1]
        daily.loc[-1] = [day, last_status['TankTemp'], last_status['ClosingStock_tc_readjusted'],
                         last_status['ClosingHeight_tc_readjusted'], day_group['Var_tc_readjusted'].sum()]
        daily.index = daily.index + 1
        daily = daily.sort_index()
    return daily

def gaussian(x, mean, sigma):
    return np.exp(-((x - mean) ** 2) / (2 * sigma ** 2))
def membership_recent(x):
    if x < 1 or x > 7:
        return 0
    elif 1 <= x <= 3:
        return 1
    elif 3 < x <= 7:
        return (7 - x) / (7 - 3)
def membership_medium(x):
    if x <= 3 or x > 19:
        return 0
    elif 3 < x < 10:
        return (x - 3) / (10 - 3)
    elif 10 <= x <= 15:
        return 1
    elif 15 < x <= 19:
        return (19 - x) / (19 - 15)
def membership_long(x):
    if x <= 15 or x > 30:
        return 0
    elif 15 < x < 25:
        return (x - 15) / (25 - 15)
    elif 25 <= x <= 30:
        return 1
def create_data(sample):
    sample = sample.sort_values(by='Day', ascending=True)
    sample['Day_Order'] = sample['Day'].rank(ascending=False)
    sample['Membership_Recent'] = sample['Day_Order'].apply(membership_recent)
    sample['Membership_Medium'] = sample['Day_Order'].apply(membership_medium)
    sample['Membership_Long'] = sample['Day_Order'].apply(membership_long)
    sample['Membership_Recent'] = sample['Membership_Recent'] / sample['Membership_Recent'].sum()
    sample['Membership_Medium'] = sample['Membership_Medium'] / sample['Membership_Medium'].sum()
    sample['Membership_Long'] = sample['Membership_Long'] / sample['Membership_Long'].sum()

    Vartc_Recent = (sample['Var_tc_readjusted'] * sample['Membership_Recent']).sum()
    Vartc_Medium = (sample['Var_tc_readjusted'] * sample['Membership_Medium']).sum()
    Vartc_Long = (sample['Var_tc_readjusted'] * sample['Membership_Long']).sum()
    ClosingHeight_tc_Recent = (sample['ClosingHeight_tc_readjusted'] * sample['Membership_Recent']).sum()
    ClosingStock_tc_Recent = (sample['ClosingStock_tc_readjusted'] * sample['Membership_Recent']).sum()
    Temp_Recent = (sample['TankTemp'] * sample['Membership_Recent']).sum()

    var_rec_med = Vartc_Recent - Vartc_Medium
    var_rec_long = Vartc_Recent - Vartc_Long
    features = [var_rec_med, var_rec_long, ClosingStock_tc_Recent, ClosingHeight_tc_Recent, Temp_Recent]
    return features

In [8]:
window_size = pd.Timedelta(days=30)
columns_sel = ['(Fuel Variance of recent period - Fuel Variance of medium period)', '(Fuel Variance of recent period - Fuel Variance of long period)', 'Inventory height of recent period', 'Likelihood']
df_test = pd.read_csv('../test_tl_AN.csv', header=0, sep=',')
tlgrouths = pd.read_csv('../tankleakage_info_AN.csv',index_col=0).reset_index(drop=True)
df_test['Time'] = pd.to_datetime(df_test['Time'])
diff = {0: 'very negative', 1: 'moderately negative', 2: 'near zero', 3: 'positive'}
pos = {0: 'very low', 1: 'low', 2: 'middle', 3: 'high', 4: 'very high'}
site, tank_id = 'D143', '1'
tank_sample_id = site + '_' + tank_id

In [9]:
model = torch.load('models/model_' + site + '.h5')
tank_df = df_test[df_test['group_id'] == tank_sample_id]
tank_info = tlgrouths[(tlgrouths['Site'] == site) & (tlgrouths['Tank'] == int(tank_id))]
startdate = tank_info.iloc[0]['StartDate']
temp_df = tank_df[tank_df['Time_DN'] > startdate]
end_date = temp_df.iloc[0]['Time'] + pd.Timedelta(days=3)
start_date = end_date - window_size
sub_df = tank_df[(tank_df['Time'] >= start_date) & (tank_df['Time'] <= end_date)]
daily = daily_format(sub_df)
feature = create_data(daily)
list_val = np.array(feature[:3])
pred = model(torch.Tensor([list_val]))
pred2 = torch.argmax(pred, 1)
pred2 = pred2.detach().numpy()

if pred2 == 0:
    res = 'non-leakage'
else:
    res = 'leakage'

rule, firerule, index_rule = load_weights.get_fire_strength(model, pred2)
numeric_pattern = r'\d+\.\d+'
numeric_values = [float(match) for match in re.findall(numeric_pattern, rule)]
conf_score = max(numeric_values)

cons, rstr = load_weights.read_rule(model)
exp = index_rule[0]
list_exp = []
for l in range(len(columns_sel) - 1):
    deg = model.layer['rules'].mf_indices[exp, l].item()
    if l == 0 or l == 1:
        descrp = diff.get(deg)
    else:
        descrp = pos.get(deg)
    ant = columns_sel[l] + " is " + descrp
    list_exp.append(ant)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_12404\3160114500.py:13: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  pred = model(torch.Tensor([list_val]))


In [15]:
reasons = []
for j in index_rule[0:1]:
    rr = 'The top firing rule is RULE ' + str(j) + ' with a firing strength of ' + str(round(firerule.get(j), 2)) + ':  \n IF '
    for l in range(len(columns_sel) - 1):
        deg = model.layer['rules'].mf_indices[j, l].item()
        if l == 0 or l == 1:
            descrp = diff.get(deg)
        else:
            descrp = pos.get(deg)
        if l == len(columns_sel) - 2:
            rr += f"*{columns_sel[l]}* is **{descrp}**"
        else:
            rr += f"*{columns_sel[l]}* is **{descrp}**  \n AND "

    temp = rstr[j]
    numeric_pattern = r'\d+\.\d+'
    numeric_values = [float(match) for match in re.findall(numeric_pattern, temp)]
    max_value = max(numeric_values)
    preds = numeric_values.index(max_value)
    if preds == 0:
        res = 'non-leakage'
    else:
        res = 'leakage'
    
    rr += f"  \n THEN the case is **{res}**"
    display(Markdown(rr))  # Print the Markdown string

    # Add a blank line for spacing
    display(Markdown(""))

    # Organize the linguistic terms
    memberships = model.fuzzified[0, :, :]
    top_values, top_indices = torch.topk(memberships.data, k=2, dim=-1)
    text = '### Feature Descriptions are:  \n'
    
    # Display the feature descriptions text
    display(Markdown(text))
    
    for i in range(memberships.size(0)):
        var = columns_sel[i]
        if i != 2:
            top1 = diff.get(top_indices[i][0].item())
            top2 = diff.get(top_indices[i][1].item())
        else:
            top1 = pos.get(top_indices[i][0].item())
            top2 = pos.get(top_indices[i][1].item())
        
        text = (
            f"*{var}* is "
            + f'**{top1}** with a membership of {str(round(top_values[i][0].item() * 100, 2))}%, and '
            + f'**{top2}** with a membership of {str(round(top_values[i][1].item() * 100, 2))}%.'
        )
        display(Markdown(text))



The top firing rule is RULE 21 with a firing strength of 0.52:  
 IF *(Fuel Variance of recent period - Fuel Variance of medium period)* is **moderately negative**  
 AND *(Fuel Variance of recent period - Fuel Variance of long period)* is **moderately negative**  
 AND *Inventory height of recent period* is **low**  
 THEN the case is **leakage**

### Feature Descriptions are:  


*(Fuel Variance of recent period - Fuel Variance of medium period)* is **moderately negative** with a membership of 98.09%, and **very negative** with a membership of 9.87%.

*(Fuel Variance of recent period - Fuel Variance of long period)* is **moderately negative** with a membership of 77.44%, and **very negative** with a membership of 26.42%.

*Inventory height of recent period* is **low** with a membership of 68.16%, and **middle** with a membership of 26.99%.